# Audio and signals

We'll finish this chapter off with a quick overview of acoustic signals: how waves propagate in an environment, and how we can reason about them as signals.

This introduction is superficial, and will mainly become relevant much later on when we go into convolutional filtering, but it may help to ground some of the principals in physical reality.

In [1]:
%run ../setup.ipynb

## Sound propagation

Let's imagine a simple recording scenario.  There is some source emitting sound signal $x_s$, perhaps a musical instrument or a human voice, and a microphone is placed 5 meters away, recording the microphone signal $x_m$.
To keep things simple, let's imagine that there are no other sounds in the environment, and there are no surfaces that could cause reflections or resonances.

If the speed of sound is some $C$ ``[meters / second]``, then it will take $5 / C$ ``[seconds]`` for sound to travel from the source to the microphone.
Put another way, the pressure we measure at time $t$ is induced by the source signal at time $t - 5/C$.
More generally, if the distance is $D$ ``[meters]``, we have the following

```{margin} Negative time offset?
Many people find the negative time offset in {eq}`distance-delay` to be counter-intuitive, and it's a common mistake to instead write $t+D/C$.
It can help to reason about what $x_m(t)$ represents: it's recording a signal that must have already occurred.
If $x_m(t)$ depended on $x_s(t + D/C)$, then it must be recording from the future, which is impossible.
```
```{math}
:label: distance-delay
x_m(t) \approx x_s(t - D/C).
```

We've written $\approx$ here, and not $=$, because the signals will not be identical: the amplitude will also change!
To see why this is the case, let's assume that the sound source is emitting uniformly in all directions.


## Intensity and loudness

Acoustic signals have many interesting properties, but perhaps the most basic is that of **intensity**: how loud is the signal?

Intensity and loudness are related, but distinct concepts.  **Intensity** refers to a physical property of the sound, namely the amplitude of the signal.
**Loudness**, on the other hand, is a *perceptual* concept, referring to how intensity is perceived by a person.

As a general rule, higher intensity means higher loudness, but it's not a one-to-one correspondance, as several other factors come into play with perception.
Humans can perceive different tones with equal intensity as having different loudness, depending on the frequency of each tone.
This phenomenon was modeled by the [Fletcher-Munson curves](https://en.wikipedia.org/wiki/Equal-loudness_contour), and later encapsulated in a more general framework of *equal loudness contours*.

A thorough discussion of psychoacoustics is beyond the scope of this text, but for now it's important to understand that loudness and intensity are distinct.

### The decibel scale (dB)


## Frequency and pitch

### Some useful facts about frequencies

- Range of human hearing
- Range of human voice
- Range of instruments
- Frequency is multiplicative, not additive!


### Harmonics